In [1]:

# Import the PlannerLLM and prompt builder utilities
from agent.model_planner import PlannerLLM, create_planner_prompt
from pathlib import Path
import json, os

PROJECT_ROOT = Path.cwd().parent
print(f"Project Root identified as: {PROJECT_ROOT}")

prompt_template_path = PROJECT_ROOT / "prompts" / "planner_prompt.txt"
few_shot_examples_path = PROJECT_ROOT / "prompts" / "few_shot_examples.json"

class ModelPlanner:
    """Small convenience wrapper matching the API used in the example.
    - generate_plan(question, prompt_template_path, few_shot_path) -> parsed plan (list/dict) or raw text
    """
    def __init__(self, model_name='google/flan-t5-small', device=None, cache_dir=None):
        self.planner = PlannerLLM(model_name=model_name, device=device, cache_dir=cache_dir)
        # We will lazily load the heavy HF model when generate_plan is called.
        self._loaded = False

    def generate_plan(self, question, prompt_template_path=prompt_template_path, few_shot_path=few_shot_examples_path, max_examples=6, num_return_sequences=3):
        # Load model if not loaded
        if not self._loaded:
            print('Loading model (this may download weights on first run)...')
            self.planner.load_model()
            self._loaded = True

        # Build the prompt using the template and few-shot examples
        prompt = create_planner_prompt(question, prompt_template_path, few_shot_path, max_examples=max_examples)

        # Ask the planner to generate plans (samples)
        candidates = self.planner.generate(prompt, max_length=512, num_return_sequences=num_return_sequences)
        # Return parsed JSONs if available; else return raw texts
        parsed = []
        for cand in candidates:
            if cand.get('parsed') is not None:
                parsed.append(cand['parsed'])
            else:
                parsed.append({'raw': cand.get('raw')})
        return parsed

    def close(self):
        self.planner.close()


Project Root identified as: /home/devcontainers/reasoning-llm-agent


In [3]:
question = "A coffee shop sold 45 lattes on Monday and 20% more lattes on Tuesday. How many lattes did they sell in total over the two days?"

# Build prompt (inspect or save preview)
prompt_text = create_planner_prompt(question, prompt_template_path, few_shot_examples_path)       
open("prompt_debug.txt","w").write(prompt_text)  # full prompt for inspection

# Generate from the model (stronger decoding)
planner = PlannerLLM(model_name="google/flan-t5-small")
planner.load_model()
candidates = planner.generate(prompt_text, max_length=512, num_return_sequences=3, temperature=0.1, top_p=1.0)
planner.close()

for i, c in enumerate(candidates, 1):
    print("=== candidate", i, "===\n", c["raw"][:1000])
    print("parsed:", c["parsed"])


=== candidate 1 ===
 a = [] b = [] for i in range(1, 10): a.append(a[i]) b.append(a[i]) c = 0 for i in range(1, 10): if a[i] == b[i]: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1 if c == 0: c += 1
parsed: []
=== candidate 2 ===
 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 + 1 a = 10 * 10 +